In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [4]:
def splitDateInColumns(df):
    for index, row in df.iterrows():
        date_object = row["Date"]
        df.at[index, 'Day'] = date_object.day
        df.at[index, 'Month'] = date_object.month
        df.at[index, 'Year'] = date_object.year

    return df

In [5]:
# Solution from : https://stackoverflow.com/questions/32854677/how-to-deal-with-multiple-date-string-formats-in-a-python-series
def timeToSeconds(timestr):
    for fmt in ("%M:%S.%f", "%S.%f", "%S:%f", "%M:%S:%f"):
        try:
            t = datetime.strptime(timestr, fmt)
            seconds = (t.minute * 60) + t.second + (t.microsecond * 1e-6)
            return round(seconds)
            break
        except ValueError:
            pass

In [6]:
def createTeams(df):
    for index, row in df.iterrows():
        person1 = row["Person 1"].upper()
        person2 = row["Person 2"].upper()
        team = (person1 + "_" + person2).replace(" ","")
        team = team.replace(",","")
        df.at[index,"Team"]  = team
    return df
    

In [7]:
def cleanColumns(df):
    #df.info()
    df = df.dropna(subset=['Date', 'Person 1', 'Person 2','split time 1','split time 2','split time 3', 'Total Time'])

    for index,row in df.iterrows():
        df.at[index,"Date"] = row["Date"] + " " + row["Time"]

    df['Date'] = pd.to_datetime(df['Date'])

    df.set_index("Date")
    df = df.sort_index()

    df.info()

    return df

In [8]:
def addTimeToDateIndex(df):
    for index,row in df.iterrows():
        timeObj = datetime.strptime(row["Time"],'%H:%M')
        df.at[index,"Date"] = datetime.datetime.combine(row["Date"],timeObj)
    return df 

In [9]:
def convertTimes(df):
    for index, row in df.iterrows():
        df.at[index, "split time 1"] = timeToSeconds(row["split time 1"])
        df.at[index, "split time 2"] = timeToSeconds(row["split time 2"])
        df.at[index, "split time 3"] = timeToSeconds(row["split time 3"])
        df.at[index, "Total Time"] = timeToSeconds(row["Total Time"])
    return df


In [10]:
def createCompetitionColumns(df):
    df = pd.get_dummies(df,columns=['Competiton Type','Competition Round'], dtype=bool)
    return df

In [12]:
def create_DeltaTimes(df):
   for index, row in df.iterrows():
      df.at[index,"D0_250"] = row["split time 1"]
      df.at[index,"D250_500"] = pd.to_numeric(row["split time 2"])-pd.to_numeric(row["split time 1"])
      df.at[index,"D500_750"] = pd.to_numeric(row["split time 3"])-pd.to_numeric(row["split time 2"])
      df.at[index,"D750_1000"] = pd.to_numeric(row["Total Time"])-pd.to_numeric(row["split time 3"])

   #print(df.head())
   return df


In [11]:
def create_tactic_columns:
    #Fast/Neutral/Slow Start?
    #Fast/Neutral/Slow  Middle?
    #Fast/Neutral/Slow End? 

SyntaxError: invalid syntax (<ipython-input-11-cc1662bc6811>, line 1)

In [13]:
def createPb_TimecurrentSeason(df):
    for index, row in df.iterrows():
        #Verkrijg alle resultaten voor deze datum en van hetzelfde jaar 
        results = df[(df["Year"] == row["Year"]) & (df["Date"] < row["Date"]) & (df["Team"]==row["Team"])].sort_values("Total Time")
        if(results.shape[0] > 0):
            bestTime = results["Total Time"].iloc[0]
            df.at[index,"BestSeasonTime"] = bestTime
        else: 
            bestTime = row["Total Time"]
            df.at[index,"BestSeasonTime"] = bestTime
    return df
    

In [ ]:
def create_Pb_Rank_currentSeason():

In [14]:
def create_Reached_FinalA(df):
    #Loop over elke row
    for index, row in df.iterrows():
        #check of ploeg in row i in de a finale komt
        dataCount = df[(df["Competition Round_FINAL A"] == True) & (df["Year"] == row["Year"]) & (df["Place"] == row["Place"]) & (df["Team"] == row["Team"])].shape[0]
        #als ploeg in de a finale geraakt
        if(dataCount >= 1):
            df.at[index,"Reached_FINAL_A"] = True
        else: 
            df.at[index,"Reached_FINAL_A"] = False        
    return df
            

In [21]:
def create_Reached_Semi(df):
    #Loop over elke row
    for index, row in df.iterrows():
        #check of ploeg in row i in de Semi komt
        dataCount = df.loc[(df["Competition Round_SEMIFINAL"] == True) & (df["Year"] == row["Year"]) & (df["Place"] == row["Place"]) & (df["Team"] == row["Team"])].shape[0]
        #als ploeg in de a finale geraakt
        if(dataCount >= 1):
            df.at[index,"Reached_SEMI"] = True
        else: 
            df.at[index,"Reached_SEMI"] = False        
    return df

In [15]:
def create_count_FinalA_currentSeason(df):
    for index, row in df.iterrows():
        #Verkrijg alle resultaten voor deze datum en van hetzelfde jaar 
        results = df[(df["Year"] == row["Year"]) & (df["Date"] < row["Date"]) & (df["Team"]==row["Team"]) & (df["Competition Round_FINAL A"] == True)].shape[0]
        df.at[index,"FinalA_count_Season"] = results
    return df

In [16]:
def getFastestTimeInRace(df):
    dfc = df.groupby("Date")["Total Time"]
    df = df.assign(N1Time=dfc.transform(min))
    return df

In [17]:
def getAverageTimeofRace(df):
    dfc = df.groupby("Date")["Total Time"]
    

In [18]:
def dropColumns(df):
    df = df.drop(columns=["Person 1", "Person 2","Wind Speed", "Wind Direction","Sex","Category"],axis=1)
    return df

In [19]:
def createAVGTIMETOP3(df):
    filterTop3= df[(df["Final Rank"] < 4)]
    filterTop3["Total Time"] = pd.to_numeric(filterTop3["Total Time"])
    groupbyRace = filterTop3.groupby("Date")["Total Time"].avg().to_frame(name="Top3Avg").reset_index()
    df = pd.merge(df,groupbyRace,how="outer")

    print(filterTop3.dtypes)
    return df
  
    


In [22]:
df = pd.read_csv("data.csv",sep=";",error_bad_lines=False)


# Datum opschonen
df = cleanColumns(df)
#df = createRaceIndex(df)
# Datum opsplitens in verschillende columns
df = splitDateInColumns(df)
#Create Teams
df = createTeams(df)
# Convert alle tijden naar seconden
convertTimes(df)
#Make columns of categorial columns
df = createCompetitionColumns(df)
df = create_Reached_FinalA(df)
df = create_Reached_Semi(df)
df = create_DeltaTimes(df)



df = create_count_FinalA_currentSeason(df)
df = createPb_TimecurrentSeason(df)
df = getFastestTimeInRace(df)

df =createAVGTIMETOP3(df)
#getAverageTimeofRace(df)
#df = dropColumns(df)
df.to_csv("K2_MEN_DATASET.csv")



/Users/BramSikkens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 0 to 542
Data columns (total 19 columns):
Date                 440 non-null datetime64[ns]
Place                440 non-null object
Time                 440 non-null object
Category             440 non-null object
Sex                  440 non-null object
Country              440 non-null object
Lane                 432 non-null float64
Person 1             440 non-null object
Person 2             440 non-null object
split time 1         440 non-null object
split time 2         440 non-null object
split time 3         440 non-null object
Total Time           440 non-null obje

AttributeError: 'SeriesGroupBy' object has no attribute 'avg'